In [1]:
import numpy as np
import itertools
import math
import scipy
from scipy import spatial
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
from matplotlib import animation
from matplotlib import transforms
from mpl_toolkits.axes_grid1 import make_axes_locatable
import xarray as xr
import dask
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import netCDF4

In [2]:
def plot_generator_paper(sample, X, Z):
    
    fz = 15*1.25
    lw = 4
    siz = 100
    XNNA = 1.25 # Abscissa where architecture-constrained network will be placed
    XTEXT = 0.25 # Text placement
    YTEXT = 0.3 # Text placement
    
    plt.rc('text', usetex=False)
    matplotlib.rcParams['mathtext.fontset'] = 'stix'
    matplotlib.rcParams['font.family'] = 'STIXGeneral'
    #mpl.rcParams["font.serif"] = "STIX"
    plt.rc('font', family='serif', size=fz)
    matplotlib.rcParams['lines.linewidth'] = lw
    
    
    cmap="RdBu_r"
    fig, ax = plt.subplots(1,1, figsize=(15,6))
    cs0 = ax.pcolor(X, Z, sample, cmap=cmap, vmin=-1.0, vmax = 1.0)
    ax.set_title("Anomalous Vertical Velocity Field Detected By ELBO")
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_xlabel("CRMs", fontsize=fz*1.5)
    ax.xaxis.set_label_coords(0.54,-0.05)
    h = ax.set_ylabel("hPa", fontsize = fz*1.5)
    h.set_rotation(0)
    ax.yaxis.set_label_coords(-0.10,0.44)
    #y_ticks = np.arange(1350, 0, -350)
    #ax.set_yticklabels(y_ticks, fontsize=fz*1.33)
    ax.tick_params(axis='x', labelsize=fz*1.33)
    ax.tick_params(axis='y', labelsize=fz*1.33)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(cs0, cax=cax)
    cbar.set_label(label=r'$\left(\mathrm{m\ s^{-1}}\right)$', rotation="horizontal", fontsize=fz*1.5, labelpad=30, y = 0.65)
    plt.show()
    #plt.savefig("/fast/gmooers/gmooers_git/CBRAIN-CAM/MAPS/CI_Figure_Data/Anomaly.pdf")
    
#plot_generator(test[0,:,:])

In [3]:
path_to_file = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-20-00000.nc'
extra_variables = xr.open_dataset(path_to_file)
lats = np.squeeze(extra_variables.LAT_20s_to_20n.values)
lons = np.squeeze(extra_variables.LON_0e_to_360e.values)

In [4]:
print(lats)

[-19.89473684 -18.         -16.10526316 -14.21052632 -12.31578947
 -10.42105263  -8.52631579  -6.63157895  -4.73684211  -2.84210526
  -0.94736842   0.94736842   2.84210526   4.73684211   6.63157895
   8.52631579  10.42105263  12.31578947  14.21052632  16.10526316
  18.          19.89473684]


In [10]:
print(lons[-80])

160.0


In [19]:
#print(int(round((lons[-30]/360.)*96.)))
#print(int(round((lons[1]/360.)*96.)))
#print(int(round((lons[20]/360.)*96.)))
#print(int(round((lons[-56]/360.)*96.)))

76
1
13
59


In [ ]:
path_to_file = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.20*'
extra_variables = xr.open_mfdataset(path_to_file)

amazon = xr.DataArray.squeeze(extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[20+96*3:96*13+20,:,:,:,10,-30]).values
sc_cloud = xr.DataArray.squeeze(extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[95+96*3:96*13+95,:,:,:,4,1]).values
african_horn = xr.DataArray.squeeze(extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[83+96*3:96*13+83,:,:,:,-6,20]).values
warm_pool = xr.DataArray.squeeze(extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[37+96*3:96*13+37,:,:,:,-11,-80]).values


In [ ]:
Max_Scalar = np.load("/fast/gmooers/Preprocessed_Data/W_Variable/Space_Time_Max_Scalar.npy")
Min_Scalar = np.load("/fast/gmooers/Preprocessed_Data/W_Variable/Space_Time_Min_Scalar.npy")

In [ ]:
amazon = np.interp(amazon, (Min_Scalar, Max_Scalar), (0, 1))
sc_cloud = np.interp(sc_cloud, (Min_Scalar, Max_Scalar), (0, 1))
african_horn = np.interp(african_horn, (Min_Scalar, Max_Scalar), (0, 1))
warm_pool = np.interp(warm_pool, (Min_Scalar, Max_Scalar), (0, 1))

In [ ]:
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_synoptic_amazon_point.npy",amazon)
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_synoptic_sc_point.npy",sc_cloud)
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_synoptic_desert_point.npy",african_horn)
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_synoptic_warm_pool_point.npy",warm_pool)

In [ ]:
path_to_file = '/DFS-L/DATA/pritchard/gmooers/Workflow/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.20*'
extra_variables = xr.open_mfdataset(path_to_file)

amazon = xr.DataArray.squeeze(extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[20+96*3:,:,:,:,10,-30]).values
sc_cloud = xr.DataArray.squeeze(extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[95+96*3:,:,:,:,4,1]).values
african_horn = xr.DataArray.squeeze(extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[83+96*3:,:,:,:,-6,20]).values
warm_pool = xr.DataArray.squeeze(extra_variables.CRM_W_LON_0e_to_360e_LAT_20s_to_20n[37+96*3:,:,:,:,-11,-80]).values


In [ ]:
amazon = np.interp(amazon, (Min_Scalar, Max_Scalar), (0, 1))
sc_cloud = np.interp(sc_cloud, (Min_Scalar, Max_Scalar), (0, 1))
african_horn = np.interp(african_horn, (Min_Scalar, Max_Scalar), (0, 1))
warm_pool = np.interp(warm_pool, (Min_Scalar, Max_Scalar), (0, 1))

In [ ]:
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_all_amazon_point.npy",amazon)
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_all_sc_point.npy",sc_cloud)
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_all_desert_point.npy",african_horn)
np.save("/fast/gmooers/Preprocessed_Data/Single_Amazon_Unaveraged/w_var_all_warm_pool_point.npy",warm_pool)